In [1]:
import pandas as pd
import numpy as np
import os
from ast import literal_eval

pd.options.mode.chained_assignment = None

In [2]:
data_path = "./data"
if os.listdir(data_path) != ['tmdb_5000_credits.csv', 'tmdb_5000_movies.csv']:
    print("[ERROR] Please download and unzip the dataset in a subdirectory './data'.")
else:
    print("[INFO] The dataset is correctly placed.")

[INFO] The dataset is correctly placed.


### Loading the data

In [3]:
tmdb_credits = pd.read_csv(os.path.join(data_path, "tmdb_5000_credits.csv"))
tmdb_credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [4]:
tmdb_movies = pd.read_csv(os.path.join(data_path, "tmdb_5000_movies.csv"))
tmdb_movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [5]:
def parse_genres(genres):
    L=[]
    for genre in genres:
        L.append(genre['name'])
    if len(L) == 0:
        return np.nan
    return L


In [6]:
movie_genres = tmdb_movies[["genres", "release_date"]]
movie_genres.head()

,genres,release_date
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2009-12-10
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",2007-05-19
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2015-10-26
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",2012-07-16
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2012-03-07


In [7]:
movie_genres["genres"] = movie_genres.genres.apply(lambda x: parse_genres(literal_eval(x)))
movie_genres.head()

,genres,release_date
0,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10
1,"[Adventure, Fantasy, Action]",2007-05-19
2,"[Action, Adventure, Crime]",2015-10-26
3,"[Action, Crime, Drama, Thriller]",2012-07-16
4,"[Action, Adventure, Science Fiction]",2012-03-07


In [8]:
movie_genres = movie_genres.dropna().explode("genres").reset_index().drop(columns="index")
movie_genres.head()

,genres,release_date
0,Action,2009-12-10
1,Adventure,2009-12-10
2,Fantasy,2009-12-10
3,Science Fiction,2009-12-10
4,Adventure,2007-05-19


In [9]:
movie_genres['release_date'] = pd.to_datetime(movie_genres['release_date'])
movie_genres['year'] = movie_genres['release_date'].apply(lambda x: x.strftime('%Y'))
movie_genres.head()

,genres,release_date,year
0,Action,2009-12-10,2009
1,Adventure,2009-12-10,2009
2,Fantasy,2009-12-10,2009
3,Science Fiction,2009-12-10,2009
4,Adventure,2007-05-19,2007


In [10]:
# Use only given years and drop TV Movies and Foreign genres

genres_to_drop = ["TV Movie","Foreign", "Mystery", "Documentary", "Animation"]
movie_genres = movie_genres[movie_genres["year"].astype(int)>1965]
movie_genres = movie_genres[movie_genres["year"].astype(int)<2017]
movie_genres = movie_genres[~movie_genres["genres"].isin(genres_to_drop)]


In [11]:
df = movie_genres.groupby(["year","genres"],  as_index=False).count()
df.head()

,year,genres,release_date
0,1966,Action,1
1,1966,Adventure,1
2,1966,Comedy,2
3,1966,Crime,2
4,1966,Drama,2


In [12]:
df['norm_per_year'] = df['release_date'] / df.groupby('year')['release_date'].transform('sum')
df.head(40)

,year,genres,release_date,norm_per_year
0,1966,Action,1,0.066667
1,1966,Adventure,1,0.066667
2,1966,Comedy,2,0.133333
3,1966,Crime,2,0.133333
4,1966,Drama,2,0.133333
5,1966,Family,1,0.066667
6,1966,History,1,0.066667
7,1966,Music,1,0.066667
8,1966,Science Fiction,1,0.066667
9,1966,Thriller,2,0.133333


In [13]:
years = list(movie_genres.year.unique())
gens = list(movie_genres.genres.unique())

index = pd.MultiIndex.from_product([years, gens], names = ["year", "genres"])

zero_df = pd.DataFrame(index = index).reset_index()
zero_df["release_date"] = 0
zero_df["norm_per_year"] = 0.0

In [14]:
zero_df.shape

(765, 4)

In [15]:
df = pd.concat([df,zero_df])

In [16]:
df.drop_duplicates(subset=['year', 'genres'], inplace=True, keep='first')

In [17]:
a = df[df.year=="1966"].copy()
a.year = "1965"
b = df[df.year=="2016"].copy()
b.year = "2017"
df = pd.concat([a,df])
df = pd.concat([df,b])

In [18]:
df.rename(columns={"release_date": "total_films"}, inplace=True)
df.head()

,year,genres,total_films,norm_per_year
0,1965,Action,1,0.066667
1,1965,Adventure,1,0.066667
2,1965,Comedy,2,0.133333
3,1965,Crime,2,0.133333
4,1965,Drama,2,0.133333


In [19]:
df.groupby("genres").count().shape

(15, 3)

In [20]:
df.sort_values(["year","genres"],inplace=True)

In [21]:
#df.to_csv("genres_first_level_stream.csv")
df_gfl = df.copy()

#### Production houses

In [22]:
pc = tmdb_movies[["genres", "release_date", "production_companies"]]
pc.head()

,genres,release_date,production_companies
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2009-12-10,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",2007-05-19,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2015-10-26,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",2012-07-16,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2012-03-07,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"


In [23]:
def parse_comp(comps):
    L=[]
    for comp in comps:
        L.append(comp['name'])
    if len(L) == 0:
        return np.nan
    return L


In [24]:
pc.head()

,genres,release_date,production_companies
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2009-12-10,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",2007-05-19,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2015-10-26,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",2012-07-16,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2012-03-07,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"


In [25]:
pc["genres"] = pc.genres.apply(lambda x: parse_genres(literal_eval(x)))
pc.head()

,genres,release_date,production_companies
0,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,"[Adventure, Fantasy, Action]",2007-05-19,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,"[Action, Adventure, Crime]",2015-10-26,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,"[Action, Crime, Drama, Thriller]",2012-07-16,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,"[Action, Adventure, Science Fiction]",2012-03-07,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"


In [26]:
pc["production_companies"] = pc["production_companies"] .apply(lambda x: parse_comp(literal_eval(x)))
pc.head()

,genres,release_date,production_companies
0,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,"[Ingenious Film Partners, Twentieth Century Fo..."
1,"[Adventure, Fantasy, Action]",2007-05-19,"[Walt Disney Pictures, Jerry Bruckheimer Films..."
2,"[Action, Adventure, Crime]",2015-10-26,"[Columbia Pictures, Danjaq, B24]"
3,"[Action, Crime, Drama, Thriller]",2012-07-16,"[Legendary Pictures, Warner Bros., DC Entertai..."
4,"[Action, Adventure, Science Fiction]",2012-03-07,[Walt Disney Pictures]


In [27]:
pc = pc.dropna().explode("production_companies").reset_index().drop(columns="index")
pc.head()

,genres,release_date,production_companies
0,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,Ingenious Film Partners
1,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,Twentieth Century Fox Film Corporation
2,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,Dune Entertainment
3,"[Action, Adventure, Fantasy, Science Fiction]",2009-12-10,Lightstorm Entertainment
4,"[Adventure, Fantasy, Action]",2007-05-19,Walt Disney Pictures


In [28]:
def clean_pc(pc):
    pc.loc[pc['production_companies'].str.startswith("Walt Disney"), "production_companies"] = "Walt Disney"
    pc.loc[pc['production_companies'].str.startswith("Paramount"), "production_companies"] = "Paramount Pictures"
    pc.loc[pc['production_companies'].str.startswith("Universal"), "production_companies"] = "Universal Pictures"
    pc.loc[pc['production_companies'].str.startswith("Twentieth"), "production_companies"] = "20th Century Studios"
    pc.loc[pc['production_companies'].str.startswith("20th"), "production_companies"] = "20th Century Studios"
    pc.loc[pc['production_companies'].str.startswith("Columbia Pictures"), "production_companies"] = "Columbia Pictures"
    pc.loc[pc['production_companies'].str.startswith("New Line"), "production_companies"] = "New Line Cinema"
    pc.loc[pc['production_companies'].str.startswith("MGM"), "production_companies"] = "Metro-Goldwyn-Mayer (MGM)"
    pc.loc[pc['production_companies'].str.startswith("Metro-Goldwyn"), "production_companies"] = "Metro-Goldwyn-Mayer (MGM)"
    pc.loc[pc['production_companies'].str.startswith("Warner"), "production_companies"] = "Warner Bros."
    return pc

In [29]:
pc = clean_pc(pc)

In [30]:
pc[pc['production_companies'].str.startswith("Walt Disney")].production_companies.unique()

array(['Walt Disney'], dtype=object)

In [31]:
pc[pc['production_companies'].str.startswith("Paramount")].production_companies.unique()

array(['Paramount Pictures'], dtype=object)

In [32]:
pc[pc['production_companies'].str.startswith("Universal")].production_companies.unique()

array(['Universal Pictures'], dtype=object)

In [33]:
pc[pc['production_companies'].str.startswith("Twentieth")].production_companies.unique()

array([], dtype=object)

In [34]:
pc[pc['production_companies'].str.startswith("Relativity")].production_companies.unique()

array(['Relativity Media', 'Relativity Sports'], dtype=object)

In [35]:
top_pc = pc.groupby("production_companies").count().sort_values("release_date",ascending=False )
top_pc_list = list(top_pc.head(10).reset_index()["production_companies"])
top_pc_list

['Warner Bros.',
 'Universal Pictures',
 'Paramount Pictures',
 'Columbia Pictures',
 '20th Century Studios',
 'New Line Cinema',
 'Walt Disney',
 'Metro-Goldwyn-Mayer (MGM)',
 'Touchstone Pictures',
 'Relativity Media']

In [36]:
pc = pc.dropna().explode("genres").reset_index().drop(columns="index")
pc.head()

,genres,release_date,production_companies
0,Action,2009-12-10,Ingenious Film Partners
1,Adventure,2009-12-10,Ingenious Film Partners
2,Fantasy,2009-12-10,Ingenious Film Partners
3,Science Fiction,2009-12-10,Ingenious Film Partners
4,Action,2009-12-10,20th Century Studios


In [37]:
pc = pc[pc["production_companies"].isin(top_pc_list)]
pc.reset_index(inplace=True)
pc.drop(columns=["index"], inplace=True)
pc

,genres,release_date,production_companies
0,Action,2009-12-10,20th Century Studios
1,Adventure,2009-12-10,20th Century Studios
2,Fantasy,2009-12-10,20th Century Studios
3,Science Fiction,2009-12-10,20th Century Studios
4,Adventure,2007-05-19,Walt Disney
...,...,...,...
5984,Drama,1983-03-24,Metro-Goldwyn-Mayer (MGM)
5985,Romance,1983-03-24,Metro-Goldwyn-Mayer (MGM)
5986,Action,1992-09-04,Columbia Pictures
5987,Crime,1992-09-04,Columbia Pictures


In [38]:
pc['release_date'] = pd.to_datetime(pc['release_date'])
pc['year'] = pc['release_date'].apply(lambda x: x.strftime('%Y'))
pc.head()

,genres,release_date,production_companies,year
0,Action,2009-12-10,20th Century Studios,2009
1,Adventure,2009-12-10,20th Century Studios,2009
2,Fantasy,2009-12-10,20th Century Studios,2009
3,Science Fiction,2009-12-10,20th Century Studios,2009
4,Adventure,2007-05-19,Walt Disney,2007


In [39]:

genres_to_drop = ["TV Movie","Foreign", "Mystery", "Documentary", "Animation"]
pc = pc[pc["year"].astype(int)>1965]
pc = pc[pc["year"].astype(int)<2017]
pc = pc[~pc["genres"].isin(genres_to_drop)]


In [40]:
df2 = pc.groupby(["year","production_companies","genres"],  as_index=False).count()
df2.head()

,year,production_companies,genres,release_date
0,1966,20th Century Studios,Adventure,1
1,1966,20th Century Studios,Comedy,1
2,1966,20th Century Studios,Crime,1
3,1966,20th Century Studios,Family,1
4,1966,20th Century Studios,Science Fiction,1


In [41]:
df2['norm_per_year'] = df2['release_date'] / df2.groupby(['year','genres'])['release_date'].transform('sum')
df2.head()

,year,production_companies,genres,release_date,norm_per_year
0,1966,20th Century Studios,Adventure,1,1.0
1,1966,20th Century Studios,Comedy,1,1.0
2,1966,20th Century Studios,Crime,1,0.5
3,1966,20th Century Studios,Family,1,1.0
4,1966,20th Century Studios,Science Fiction,1,1.0


In [42]:
years = list(pc.year.unique())
gens = list(pc.genres.unique())
comps = list(pc.production_companies.unique())

index = pd.MultiIndex.from_product([years, gens, comps], names = ["year", "genres", "production_companies"])

zero_df = pd.DataFrame(index = index).reset_index()
zero_df["release_date"] = 0
zero_df["norm_per_year"] = 0.0

In [43]:
zero_df

,year,genres,production_companies,release_date,norm_per_year
0,2009,Action,20th Century Studios,0,0.0
1,2009,Action,Walt Disney,0,0.0
2,2009,Action,Columbia Pictures,0,0.0
3,2009,Action,Warner Bros.,0,0.0
4,2009,Action,Paramount Pictures,0,0.0
...,...,...,...,...,...
7645,1967,Music,New Line Cinema,0,0.0
7646,1967,Music,Metro-Goldwyn-Mayer (MGM),0,0.0
7647,1967,Music,Universal Pictures,0,0.0
7648,1967,Music,Relativity Media,0,0.0


In [44]:
# We must have all genres in all years, so we populate 0 where missing
df3 = pd.concat([df2,zero_df])
df3 = df3.drop_duplicates(subset=['year', 'genres','production_companies'], keep='first')

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [45]:
df3.shape

(7650, 5)

In [46]:
a = df3[df3.year=="1966"].copy()
a.year = "1965"
b = df3[df3.year=="2016"].copy()
b.year = "2017"
df3 = pd.concat([a,df3])
df3 = pd.concat([df3,b])

In [47]:
df3.rename(columns={"release_date": "total_films"}, inplace=True)
df3.head()

,genres,norm_per_year,production_companies,total_films,year
0,Adventure,1.0,20th Century Studios,1,1965
1,Comedy,1.0,20th Century Studios,1,1965
2,Crime,0.5,20th Century Studios,1,1965
3,Family,1.0,20th Century Studios,1,1965
4,Science Fiction,1.0,20th Century Studios,1,1965


In [48]:
df3.sort_values(["year","genres", "production_companies"],inplace=True)
df3.head()

,genres,norm_per_year,production_companies,total_films,year
7350,Action,0.0,20th Century Studios,0,1965
7352,Action,0.0,Columbia Pictures,0,1965
7356,Action,0.0,Metro-Goldwyn-Mayer (MGM),0,1965
7355,Action,0.0,New Line Cinema,0,1965
7354,Action,0.0,Paramount Pictures,0,1965


In [49]:
#df3.to_csv("production_companies_second_level_stream.csv")

In [50]:
df_pcsl = df3.copy()

In [51]:
### Loading the data

tmdb_credits = pd.read_csv(os.path.join(data_path, "tmdb_5000_credits.csv"))
tmdb_credits.head(2)

tmdb_movies = pd.read_csv(os.path.join(data_path, "tmdb_5000_movies.csv"))
tmdb_movies.head(2)

def parse_genres(genres):
    L=[]
    for genre in genres:
        L.append(genre['name'])
    if len(L) == 0:
        return np.nan
    return L
def parse_comp(comps):
    L=[]
    for comp in comps:
        L.append(comp['name'])
    if len(L) == 0:
        return np.nan
    return L

pc = tmdb_movies[["production_companies", "release_date"]]
pc.head()

pc["production_companies"] = pc["production_companies"] .apply(lambda x: parse_comp(literal_eval(x)))
pc.head()

pc = pc.dropna().explode("production_companies").reset_index().drop(columns="index")
pc.head()
pc = clean_pc(pc)
top_pc = pc.groupby("production_companies").count().sort_values("release_date",ascending=False )
top_pc_list = list(top_pc.head(10).reset_index()["production_companies"])
top_pc_list
#top_pc

pc = pc[pc["production_companies"].isin(top_pc_list)]
pc.reset_index(inplace=True)
pc.drop(columns=["index"], inplace=True)
pc

pc['release_date'] = pd.to_datetime(pc['release_date'])
pc['year'] = pc['release_date'].apply(lambda x: x.strftime('%Y'))
pc.head()

pc = pc[pc["year"].astype(int)>1965]
pc = pc[pc["year"].astype(int)<2017]

df = pc.groupby(["year","production_companies"],  as_index=False).count()
df.head()

df['norm_per_year'] = df['release_date'] / df.groupby('year')['release_date'].transform('sum')
df.head(40)

years = list(pc.year.unique())
#years.append("2017")
#years.append("1965")
#a.insert(len(a),5)
pcs = list(pc.production_companies.unique())

index = pd.MultiIndex.from_product([years,pcs], names = ["year", "production_companies"])

zero_df = pd.DataFrame(index = index).reset_index()
zero_df["release_date"] = 0
zero_df["norm_per_year"] = 0.00

zero_df.shape

df = pd.concat([df,zero_df])

df.drop_duplicates(subset=['year', 'production_companies'], inplace=True, keep='first')

a = df[df.year=="1966"].copy()
a.year = "1965"
b = df[df.year=="2016"].copy()
b.year = "2017"
df = pd.concat([a,df])
df = pd.concat([df,b])

df.rename(columns={"release_date": "total_films"}, inplace=True)
df.head()

df.groupby("production_companies").count().shape

#df[df.norm_per_year==0]["norm_per_year"] = 0.1

df.head()

df.sort_values(["year","production_companies"],inplace=True)

df.head(30)

#df.to_csv("production_companies_first_level_stream.csv")
df_pcfl = df.copy()

#### Genres

pc = tmdb_movies[["genres", "release_date", "production_companies"]]
pc.head()

def parse_comp(comps):
    L=[]
    for comp in comps:
        L.append(comp['name'])
    if len(L) == 0:
        return np.nan
    return L


pc.head()

pc["genres"] = pc.genres.apply(lambda x: parse_genres(literal_eval(x)))
pc.head()

pc["production_companies"] = pc["production_companies"] .apply(lambda x: parse_comp(literal_eval(x)))
pc.head()

pc = pc.dropna().explode("production_companies").reset_index().drop(columns="index")
pc.head()
pc = clean_pc(pc)
top_pc = pc.groupby("production_companies").count().sort_values("release_date",ascending=False )
top_pc_list = list(top_pc.head(10).reset_index()["production_companies"])
top_pc_list

pc = pc.dropna().explode("genres").reset_index().drop(columns="index")
pc.head()

pc = pc[pc["production_companies"].isin(top_pc_list)]
pc.reset_index(inplace=True)
pc.drop(columns=["index"], inplace=True)
pc

pc['release_date'] = pd.to_datetime(pc['release_date'])
pc['year'] = pc['release_date'].apply(lambda x: x.strftime('%Y'))
pc.head()


genres_to_drop = ["TV Movie","Foreign", "Mystery", "Documentary", "Animation"]
pc = pc[pc["year"].astype(int)>1965]
pc = pc[pc["year"].astype(int)<2017]
pc = pc[~pc["genres"].isin(genres_to_drop)]


df2 = pc.groupby(["year","production_companies","genres"],  as_index=False).count()
df2.head()

df2['norm_per_year'] = df2['release_date'] / df2.groupby(['year','production_companies'])['release_date'].transform('sum')
df2.head()

years = list(pc.year.unique())
gens = list(pc.genres.unique())
comps = list(pc.production_companies.unique())

index = pd.MultiIndex.from_product([years, gens, comps], names = ["year", "genres", "production_companies"])

zero_df = pd.DataFrame(index = index).reset_index()
zero_df["release_date"] = 0
zero_df["norm_per_year"] = 0.0

zero_df

# We must have all genres in all years, so we populate 0 where missing
df3 = pd.concat([df2,zero_df])
df3 = df3.drop_duplicates(subset=['year', 'genres','production_companies'], keep='first')

df3.shape

a = df3[df3.year=="1966"].copy()
a.year = "1965"
b = df3[df3.year=="2016"].copy()
b.year = "2017"
df3 = pd.concat([a,df3])
df3 = pd.concat([df3,b])

df3.rename(columns={"release_date": "total_films"}, inplace=True)
df3.head()

df3.sort_values(["year","production_companies", "genres"],inplace=True)
df3.head()

#df3.to_csv("genres_second_level_stream.csv")
df_gsl = df3.copy()


C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:164: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [52]:
df_pcfl

,year,production_companies,total_films,norm_per_year
0,1965,20th Century Studios,1,0.250000
1,1965,Columbia Pictures,1,0.250000
496,1965,Metro-Goldwyn-Mayer (MGM),0,0.000000
495,1965,New Line Cinema,0,0.000000
494,1965,Paramount Pictures,0,0.000000
...,...,...,...,...
58,2017,Relativity Media,0,0.000000
59,2017,Touchstone Pictures,0,0.000000
360,2017,Universal Pictures,10,0.217391
361,2017,Walt Disney,8,0.173913


In [53]:
df_gfl["code"] = "gfl"
df_gsl["code"] = "gsl"
df_pcfl["code"]="pcfl"
df_pcsl["code"]="pcsl"

In [54]:
end_df = pd.concat([df_gfl,df_gsl,df_pcfl,df_pcsl])

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [55]:
end_df.to_csv("all_data_stream.csv")